In [31]:
import json
import random
from eval import Evaluate
import torch
eval_f = Evaluate()

In [32]:
def get_original_samples(path, is_unk):
    abstracts = {}
    with open(path) as f:
        for line in f:
            j = json.loads(line.strip())
            if not is_unk:
                abstracts[j["title"]] = j["abstract"]
            else:
                abstracts[j["abstract"]] = j["title"]
    return abstracts 

In [33]:
def get_generated_samples(path):
    abstracts = {}
    with open(path) as f:
        for line in f:
            j = json.loads(line.strip())
            abstracts[j["original"]] = j["generated"]
    return abstracts 

In [34]:
def get_abstract_metrics(gen, org):
    ref = {0 : [org]}
    cand = {0 : gen}
    final_scores = eval_f.evaluate(live=True, cand=cand, ref=ref)
    return final_scores

In [57]:
original_abstracts = get_original_samples("acl-dataset/acl_original.txt", is_unk=False)
original_unk = get_original_samples("acl-dataset/acl-UNK-abstracts.txt", is_unk=True)
generated = get_generated_samples("acl-dataset/WEPGen/ACL-WE-Topics-Structure-IntraAttention-Polishing-BLEU-4=2.733333.txt")

In [58]:
ratings = []
c = 0
for key in generated.keys():
    original_abs_UNK = key
    title = original_unk[original_abs_UNK]
    org = original_abstracts[title]
    gen = generated[key]
    metrics = get_abstract_metrics(gen, org)
    ratings.append((title, org, gen, metrics.values()))
    c += 1
    
    if c % 200 == 0:
        print("Done {}".format(c))

Done 200
Done 400
Done 600
Done 800
Done 1000


In [59]:
bleu_samples = sorted(ratings, reverse=True, key=lambda x: sum(list(x[-1])[:4]))[:100]
meteor_samples = sorted(ratings, reverse=True, key=lambda x: list(x[-1])[4])[:100]
rouge_samples = sorted(ratings, reverse=True, key=lambda x: list(x[-1])[5])[:100]
combined_samples = sorted(ratings, reverse=True, key=lambda x: sum(x[-1]))[:100]

bleu_set = set([b[:3] for b in bleu_samples])
meteor_set = set([b[:3] for b in meteor_samples])
rouge_set = set([b[:3] for b in rouge_samples])
combined_set = set([b[:3] for b in combined_samples])

A = bleu_set.intersection(meteor_set)
B = A.intersection(rouge_set)
final_set = list(B.intersection(combined_set))
len(final_set)

29

In [60]:
qualitative = {}
i = 0
for title, _, generated in final_set:
    qualitative[i] = (title, generated)
    i += 1

In [61]:
with open("acl-dataset/Qualitative/W.txt", "w") as f:
    f.write(json.dumps(qualitative))